# 画像認識による乗り物分類レシピ

https://axross-recipe.com/recipes/1

### Flickr API の準備

以下の３つの乗り物の画像を分類する
- 電車
- 車
- 自転車

In [15]:
import glob
import os
import time

from urllib.request import urlretrieve
from flickrapi import FlickrAPI
import numpy as np
from PIL import Image
from sklearn import model_selection

In [13]:
# APIキー情報
FLICKR_KEY = os.environ['FLICKR_KEY']
FLICKR_SECRET = os.environ['FLICKR_SECRET']

# 乗り物の名前
VEHICLES = ["train", "car", "bicycle"]

# 乗り物をループしてデータを取得
for vehicle in VEHICLES:
    # 保存フォルダの指定
    save_dir = os.path.join("datasets", vehicle)
    os.makedirs(save_dir, exist_ok=True)

    # Flickr APIの初期化
    flickr = FlickrAPI(FLICKR_KEY, FLICKR_SECRET, format="parsed-json")
    
    # 乗り物の名前を指定して100件の画像情報を取得
    result = flickr.photos.search(text=vehicle,
                                                      per_page=100,
                                                      media="photos",
                                                      sort="relavance",
                                                      safe_search=1,
                                                      extras="url_q, licence")
    
    # 画像情報から実際の画像ファイルを取得
    photos = result["photos"]
    for photo in photos["photo"]:
        # 画像のURL
        url_q = photo["url_q"]
        # 画像のダウンロード先
        filepath = os.path.join(save_dir, photo["id"] + ".jpg")
        # 画像を指定したパスにダウンロードして保存
        urlretrieve(url_q, filepath)
        # クローリング先のサーバーに負荷を与えない
        time.sleep(1)

### データセットを交差検証用に分割する

In [18]:
# 乗り物の名前
VEHICLES = ["train", "car", "bicycle"]
# データセットのフォルダパス
DATASET_DIR = os.path.join("datasets")

datasets = []
labels = []

for index, label in enumerate(VEHICLES):
    # 画像の読み込み
    photos_dir = os.path.join(DATASET_DIR, label)
    filepaths = glob.glob(os.path.join(photos_dir, "*.jpg"))
    
    # 画像を順次処理してデータセットを作成
    for i, filepath in enumerate(filepaths):
        # 各乗り物のデータを100に揃える
        if i >= 100:
            break
        
        # 画像の読み込み
        image = Image.open(filepath)
        # RGBの3色に変換
        image = image.convert("RGB")
        # 画像のサイズを統一
        image = image.resize((50, 50))
        # 画像を数値の配列に変換
        dataset = np.asarray(image)
        # データセットを追加
        datasets.append(dataset)
        # ラベルを追加
        labels.append(index)

# Tensorflow がデータを処理しやすいように numpy の array に変換
datasets = np.array(datasets)
labels = np.array(labels)

# データセットとラベルの両方を学習データとテストデータに分類
dataset_train, dataset_test, label_train, label_test = model_selection.train_test_split(datasets, labels)

# 分類したデータをファイルに保存
data = (dataset_train, dataset_test, label_train, label_test)
np.save(os.path.join(DATASET_DIR, "vehicle.npy"), data)

### モデルのトレーニング

In [20]:
import keras
from keras.layers import (Activation, Conv2D, Dense, Dropout, Flatten, MaxPooling2D)
from keras.models import Sequentialfrom keras.utils import np_utils

In [21]:
# 乗り物の名前
VEHICLES = ["train", "car", "bicycle"]
# データセットのフォリダパス
DATASET_DIR = os.path.join("datasets")

# データセットの読み込み
dataset_train, dataset_test, label_train, label_test = np.load(os.path.join(DATASET_DIR, "vehicle.npy"), allow_pickle=True)

In [22]:
# 画素を0〜1の範囲に変換（正規化）
dataset_train = dataset_train.astype("float") / 256
dataset_test = dataset_test.astype("float") / 256

In [24]:
# ラベルを one-hotエンコーディング
label_train = np_utils.to_categorical(label_train, len(VEHICLES))
label_test = np_utils.to_categorical(label_test, len(VEHICLES))

In [31]:
# モデルの定義
model = Sequential()
model.add(Conv2D(32, (3, 3), padding="same", input_shape=dataset_train.shape[1:]))

model.add(Activation("relu"))
model.add(Conv2D(32, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dropout(0.25))
model.add(Dense(3))
model.add(Activation("softmax"))

# RMSprop最適化関数
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
model.compile(loss="categorical_crossentropy",
                          optimizer=opt,
                          metrics=["accuracy"])

# モデルのトレーニング
model.fit(dataset_train, label_train, batch_size=32, epochs=100)

# モデルの保存
model.save(os.path.join(DATASET_DIR, "vehicle_cnn.h5"))

Epoch 1/100
224/224 [==============================] - 1s 6ms/step - loss: 1.1167 - accuracy: 0.3616
Epoch 2/100
224/224 [==============================] - 1s 5ms/step - loss: 1.0922 - accuracy: 0.3705
Epoch 3/100
224/224 [==============================] - 1s 5ms/step - loss: 1.0724 - accuracy: 0.4152
Epoch 4/100
224/224 [==============================] - 1s 5ms/step - loss: 1.0601 - accuracy: 0.4464
Epoch 5/100
224/224 [==============================] - 1s 5ms/step - loss: 1.0412 - accuracy: 0.4955
Epoch 6/100
224/224 [==============================] - 1s 5ms/step - loss: 1.0297 - accuracy: 0.5179
Epoch 7/100
224/224 [==============================] - 1s 5ms/step - loss: 1.0030 - accuracy: 0.5134
Epoch 8/100
224/224 [==============================] - 1s 5ms/step - loss: 0.9834 - accuracy: 0.5045
Epoch 9/100
224/224 [==============================] - 1s 5ms/step - loss: 0.9474 - accuracy: 0.6071
Epoch 10/100
224/224 [==============================] - 1s 5ms/step - loss: 0.9622 - accura

224/224 [==============================] - 1s 5ms/step - loss: 0.2074 - accuracy: 0.9241
Epoch 82/100
224/224 [==============================] - 1s 5ms/step - loss: 0.1112 - accuracy: 0.9777
Epoch 83/100
224/224 [==============================] - 1s 5ms/step - loss: 0.1553 - accuracy: 0.9554
Epoch 84/100
224/224 [==============================] - 1s 5ms/step - loss: 0.1353 - accuracy: 0.9554
Epoch 85/100
224/224 [==============================] - 1s 4ms/step - loss: 0.1143 - accuracy: 0.9777
Epoch 86/100
224/224 [==============================] - 1s 5ms/step - loss: 0.1226 - accuracy: 0.9643
Epoch 87/100
224/224 [==============================] - 1s 5ms/step - loss: 0.1501 - accuracy: 0.9420
Epoch 88/100
224/224 [==============================] - 1s 5ms/step - loss: 0.0981 - accuracy: 0.9777
Epoch 89/100
224/224 [==============================] - 1s 5ms/step - loss: 0.1004 - accuracy: 0.9777
Epoch 90/100
224/224 [==============================] - 1s 5ms/step - loss: 0.1058 - accuracy: 

### モデルのテストと評価

In [32]:
scores = model.evaluate(dataset_test, label_test, verbose=1)

print("LOSS: " + str(scores[0]))
print("Accuracy: " + str(scores[1]))

75/75 [==============================] - 0s 1ms/step
LOSS: 1.3743815008799236
Accuracy: 0.6266666650772095


### VGG16 を使った転移学習

元となる学習済みのモデルの層をフリーズ（再学習しない）して、学習済みモデルの最後に新しい層を追加していく。
今回は、VGG16 の最後に３つの乗り物のクラスで定義された層を追加していく。

In [34]:
from keras.layers import (Activation, Conv2D, Dense, Dropout, Flatten,
                          MaxPooling2D)
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.applications import VGG16

In [36]:
# VGG16モデルの読み込み（ImgaeNet）
vgg_model = VGG16(weights="imagenet", include_top=False, input_shape=dataset_train.shape[1:])

# 新しいモデルの定義
new_model = Sequential()
new_model.add(Flatten(input_shape=vgg_model.output_shape[1:]))
new_model.add(Dense(256, activation="relu"))
new_model.add(Dropout(0.5))
new_model.add(Dense(len(VEHICLES), activation="softmax"))

# VGG16と新しい層を結合
# VGG16モデルの出力を新しいモデルに渡す
model = Model(inputs=vgg_model.input, outputs=new_model(vgg_model.output))

# モデルの層を確認
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 50, 50, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 50, 50, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 50, 50, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 25, 25, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 25, 25, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 25, 25, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0   

In [38]:
# VGG16モデルの最初の18層をフリーズ
# VGG16の層は再学習させずに新しく追加した最後の層だけを学習
for layer in model.layers[:18]:
    layer.trainable = False

# RMSprop最適化関数
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
model.compile(loss="categorical_crossentropy",
                          optimizer=opt,
                          metrics=["accuracy"])

# モデルのトレーニング
model.fit(dataset_train, label_train, batch_size=32, epochs=30)

Epoch 1/30
224/224 [==============================] - 1s 5ms/step - loss: 1.4870 - accuracy: 0.3080
Epoch 2/30
224/224 [==============================] - 1s 4ms/step - loss: 1.2873 - accuracy: 0.3438
Epoch 3/30
224/224 [==============================] - 1s 4ms/step - loss: 1.2401 - accuracy: 0.3571
Epoch 4/30
224/224 [==============================] - 1s 4ms/step - loss: 1.2133 - accuracy: 0.3348
Epoch 5/30
224/224 [==============================] - 1s 4ms/step - loss: 1.1547 - accuracy: 0.4330
Epoch 6/30
224/224 [==============================] - 1s 4ms/step - loss: 1.1819 - accuracy: 0.3661
Epoch 7/30
224/224 [==============================] - 1s 4ms/step - loss: 1.1151 - accuracy: 0.3884
Epoch 8/30
224/224 [==============================] - 1s 4ms/step - loss: 1.1101 - accuracy: 0.4286
Epoch 9/30
224/224 [==============================] - 1s 5ms/step - loss: 1.0975 - accuracy: 0.4554
Epoch 10/30
224/224 [==============================] - 1s 5ms/step - loss: 1.0919 - accuracy: 0.4509

In [39]:
# モデルの評価
scores = model.evaluate(dataset_test, label_test, verbose = 1)
print("Loss: " + str(scores[0]))
print("Accuracy: " + str(scores[1]))

75/75 [==============================] - 0s 5ms/step
Loss: 0.8703115359942118
Accuracy: 0.6666666865348816


In [40]:
# モデルの再トレーニング
model.fit(dataset_train, label_train, batch_size=32, epochs=10)

Epoch 1/10
224/224 [==============================] - 1s 4ms/step - loss: 0.8128 - accuracy: 0.6250
Epoch 2/10
224/224 [==============================] - 1s 4ms/step - loss: 0.7667 - accuracy: 0.6830
Epoch 3/10
224/224 [==============================] - 1s 4ms/step - loss: 0.7408 - accuracy: 0.6830
Epoch 4/10
224/224 [==============================] - 1s 4ms/step - loss: 0.7344 - accuracy: 0.6875
Epoch 5/10
224/224 [==============================] - 1s 4ms/step - loss: 0.7453 - accuracy: 0.6562
Epoch 6/10
224/224 [==============================] - 1s 4ms/step - loss: 0.7329 - accuracy: 0.6652
Epoch 7/10
224/224 [==============================] - 1s 4ms/step - loss: 0.7172 - accuracy: 0.7232
Epoch 8/10
224/224 [==============================] - 1s 5ms/step - loss: 0.7163 - accuracy: 0.7321
Epoch 9/10
224/224 [==============================] - 1s 5ms/step - loss: 0.6938 - accuracy: 0.7232
Epoch 10/10
224/224 [==============================] - 1s 5ms/step - loss: 0.6725 - accuracy: 0.7545

In [41]:
# モデルの評価
scores = model.evaluate(dataset_test, label_test, verbose = 1)
print("Loss: " + str(scores[0]))
print("Accuracy: " + str(scores[1]))

75/75 [==============================] - 0s 4ms/step
Loss: 0.8402052434285482
Accuracy: 0.6800000071525574
